
### 1. TensorDataset과 DataLoader

- 입력 데이터를 쉽게 처리하고, 배치 단위로 잘러서 학습할 수 있게 도와주는 모듈
- **Dataset** : 학습시 사용하는 feature와 target의 pair로 이루어짐. 
    - 아래에서 코드에서는 TensorDataset을 사용하여 Dataset 인스턴스를 생성했지만, 이미지의 사례와 같이 Dataset 클래스를 상속받아서 커스텀 인스턴스를 생성하는 형태로 많이 사용

- **DataLoader**: 학습 시 각 인스턴스에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)를 생성

![data loader](https://sebastianraschka.com/images/blog/2022/datapipes/loader-flow.png)


- **Sample code**
```
from torch.utils.data import  TensorDataset, DataLoader
```
```
# X,y로 분할한 데이터를 tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)
```
```
# tensor를 TensorDataset으로 생성 - X와 y가 짝으로 이루어짐
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
```
```
# DataLoader 형태로 생성
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
```

- **DataLoader가 하는 역할**
    - shuffling
    - batch ...
    
![data loader](https://sebastianraschka.com/images/blog/2022/datapipes/dataflow-good.png)



### 2. Device 설정
- 일반적으로 인공신경망의 학습은 (가능하다면) GPU를 사용하는 것이 바람직함
    - Colab Runtime 설정 변경
- GPU를 사용하여 학습을 진행하도록 명시적으로 작성 필요
- 연산 유형에 따라 GPU에서 수행이 불가능한 경우도 존재하는데, 그럴 경우도 마찬가지로 명시적으로 어떤 프로세서에서 연산을 수행해야하는지 코드로 작성해야함 

```
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NeuralNetwork().to(device)
```


### 3. 신경망 생성

- **torch.nn 패키지**는 신경망 생성 및 학습 시 설정해야하는 다양한 기능을 제공

```
import torch.nn as nn
```
- 신경망을 **nn.Module**을 상속받아 정의
    - __ __init__ __(): 신경망에서 사용할 layer를 초기화하는 부분
    - __forward()__: feed foward 연산 수행 시, 각 layer의 입출력이 어떻게 연결되는지를 지정

```
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

```


### 4. Model compile

- 학습 시 필요한 정보들(loss function, optimizer)을 선언
- 일반적으로 loss와 optimizer는 아래와 같이 변수로 선언하고, 변수를 train/test 시 참고할 수 있도록 매개변수로 지정해줌 

```
learning_rate = 0.01
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
```


### 5. Train
- **신경망의 학습과정**을 별도의 함수로 구성하는 것이 일반적
    - feed forward -> loss -> error back propagation -> (log) -> (반복)

```
def train_loop(train_loader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

### 6. Test

- 학습과정과 비슷하나 error back propagate하는 부분이 없음
    - feed forward -> loss ->  (log) -> (반복)

```
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")


### 7. Iteration
- 신경망 학습은 여러 epochs을 반복해서 수행하면서 모델을 구성하는 최적의 파라미터를 찾음
- 지정한 epochs 수만큼 **학습**과정과 **평가**과정을 반복하면서, 모델의 성능(loss, accuracy 등)을 체크함

```
epochs = 10
for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")
```

## 8. Basic Neural Network
- Iris dataset을 사용하여 꽃의 품종을 구분하는 분류기를 구현한다.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

iris = load_iris() # numpy로 받아서 dataframe으로 변환
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

y = df['label']
X = df.drop(['label'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state = 42, stratify = y)

In [8]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.int64)
y_test = torch.tensor(y_test, dtype = torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 10 ,shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 10 ,shuffle = True)

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(4,16)
        self.hidden_layer1 = nn.Linear(16,32)
        self.output_layer = nn.Linear(32,3)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.output_layer(out)
        return out

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.is_available())
print(f'device = {device}')

model = NeuralNetwork().to(device)

False
device = cpu


In [13]:
lr = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [14]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)
    
    for batch, (X,y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss : {loss:>7f} [{current:>5d}]/{size:5d}')

In [17]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset) 
    num_batches = len(test_loader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f'Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:8f}\n')

In [18]:
epochs = 10

for i in range(epochs):
    print(f"Epochs {i+1} \n-----------------------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model,loss)
print("Done!")

Epochs 1 
-----------------------------------------
loss : 1.066060 [    0]/  112
loss : 1.030597 [   10]/  112
loss : 1.007098 [   20]/  112
loss : 1.034640 [   30]/  112
loss : 0.953362 [   40]/  112
loss : 0.937334 [   50]/  112
loss : 1.009688 [   60]/  112
loss : 0.967265 [   70]/  112
loss : 1.003079 [   80]/  112
loss : 0.958654 [   90]/  112
loss : 0.941158 [  100]/  112
loss : 0.888004 [   22]/  112
Test Error : 
 Accuracy : 65.8%, Avg Loss : 0.937880

Epochs 2 
-----------------------------------------
loss : 0.910976 [    0]/  112
loss : 0.988739 [   10]/  112
loss : 0.946055 [   20]/  112
loss : 0.874511 [   30]/  112
loss : 0.923309 [   40]/  112
loss : 0.871425 [   50]/  112
loss : 0.911081 [   60]/  112
loss : 0.915385 [   70]/  112
loss : 0.903536 [   80]/  112
loss : 0.956071 [   90]/  112
loss : 0.915664 [  100]/  112
loss : 0.746812 [   22]/  112
Test Error : 
 Accuracy : 65.8%, Avg Loss : 0.879445

Epochs 3 
-----------------------------------------
loss : 0.866953 